# RT Assignment

## Robot behavior ##

In [1]:
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox, interactive, widgets
from move_base_msgs.msg import MoveBaseActionGoal, MoveBaseActionFeedback
from geometry_msgs.msg import PointStamped, Twist
from tf.transformations import quaternion_matrix
from matplotlib.animation import FuncAnimation
import ipywidgets as widgets
from sensor_msgs.msg import LaserScan
from actionlib_msgs.msg import GoalID
from IPython.display import display
from nav_msgs.msg import Odometry

import matplotlib.pyplot as plt
import ipywidgets as widgets
import jupyros as jr
import numpy as np
import datetime
import rospy 
import time
import tf

global SetX, SetY, XGoal, YGoal, goalID
global notAchieved
global outputSet
global achieved
global x_vel
global z_vel

collisionAssistant = False
AssistedDrive = False
Navigation = False
goalStatus = False

notAchieved = 0
achieved = 0

In [2]:
#function called everytime the third modality is chosen for enabling the collision avoidence
def CollisionAvoidanceCallback(msg):
    global Vel, collisionAssistant, x_vel, y_vel   
    
    if collisionAssistant == True:        
        Vel = Twist()
        
        right = min(msg.ranges[0:79])
        front = min(msg.ranges[340:379])
        left = min(msg.ranges[640:719])
        
        Vel.linear.x = x_vel;
        Vel.angular.z = z_vel;

        if front < 0.5:
            Vel.linear.x = 0
        elif left < 0.5:
            Vel.angular.z = -1
        elif right < 0.5:
            Vel.angular.z = 1
        else:
            pass

        pubVel.publish(Vel) #publish the corrected velocity for avoiding collisions

In [3]:
def CurrentGoal(msg):
    
    global xG, yG
    xG = msg.goal.target_pose.pose.position.x;
    yG = msg.goal.target_pose.pose.position.y;


In [4]:
def GoalStatus(msg):
    
    global goalStatus, xG, yG
    global achieved
    
    goalID = msg.status.goal_id.id
       
    if goalStatus == True:
        if abs(msg.feedback.base_position.pose.position.x - xG) <= 0.5 and abs(msg.feedback.base_position.pose.position.y - yG)<= 0.5:
            achieved += 1
            print("Goal Reached!")
            goalStatus = False 

In [5]:
def CancelGoal(b):
    global goalStatus, notAchieved
    if goalStatus == False:
        print("No goal set!")
        notAchieved = notAchieve
    else:
        with outputCanc:
            lastGoal = GoalID()
            pubCancel.publish(lastGoal)
            goalStatus = False
            print("Goal Canceled")
            notAchieved += 1

In [6]:
def SetGoal(b):    
            
    global goalStatus            
    with outputSet:
                
        if goalStatus == True:
                    
            print("Goal already set")
                    
        else:   
                    
            XGoal = float(SetX.value)
            YGoal = float(SetY.value)
            my_goal = MoveBaseActionGoal()

            my_goal.goal.target_pose.header.frame_id = "map"
            my_goal.goal.target_pose.pose.orientation.w = 1

            my_goal.goal.target_pose.pose.position.x = XGoal
            my_goal.goal.target_pose.pose.position.y = YGoal 

            pubGoal.publish(my_goal)
                    
            goalStatus = True

In [7]:
# definition of the Buttons for SetGoal and CancGoal
bSet = widgets.Button(description='SetGoal')
bCanc = widgets.Button(description='CancGoal')

outputSet = widgets.Output()
outputCanc = widgets.Output()

bSet.on_click(SetGoal)
bCanc.on_click(CancelGoal)

In [8]:
def AutonomousNavigation(b):
    
    global SetX, SetY   
    with output1:   
        print("Autonomous Navigation!")
    
        SetX = widgets.FloatText(
        description='XValue:',
        disabled=False
        )
        
        SetY = widgets.FloatText(
        description='YValue:',
        disabled=False
        )
        display(SetX, SetY, bSet, outputSet)
        display(bCanc, outputCanc)
               

In [9]:
def GetVel(V):
    global x_vel, z_vel
    x_vel = V.linear.x;
    z_vel = V.angular.z;
subVel = rospy.Subscriber('/cmd_vel', Twist, GetVel)

In [10]:
# Definition of the buttons  for the manual drive and of the checkbox for able and disable the assisted drive
ButtonFront = widgets.Button(description = "Front")        
ButtonBack = widgets.Button(description = "Back")       
ButtonLeft = widgets.Button(description = "Left")        
ButtonRight = widgets.Button(description = "Right")      
ButtonFrontLeft = widgets.Button(description = "FrontLeft")        
ButtonFrontRight = widgets.Button(description = "FrontRight")
ButtonStop = widgets.Button(description = "Stop")
ButtonIncreaseV = widgets.Button(description = "IncreaseV")
ButtonDecreaseV = widgets.Button(description = "DecreaseV")
assistedDrive = widgets.Checkbox(
    value = False,
    description="Assisted drive",
    disabled= False,
    indent = False
)

In [11]:
def RobotNavigation(b):
    
    print("Robot Navigation!")
    display(assistedDrive)
    display(ButtonFront, ButtonBack, ButtonRight, ButtonStop, ButtonFrontLeft, ButtonFrontRight)
    display(ButtonIncreaseV, ButtonDecreaseV)
    
    with output2:
        
        global x_vel, z_vel
        
        def setAssisted(b):
            global collisionAssistant
            if collisionAssistant == False:
                collisionAssistant = True
            else:
                collisionAssistant = False
        
        def Front(b):
            Vel = Twist()
            Vel.linear.x = 0.5
            Vel.angular.z = 0
            pubVel.publish(Vel)
            
        def Back(b):
            Vel = Twist()
            Vel.linear.x = -0.5
            Vel.angular.z = 0
            pubVel.publish(Vel)
            
        def Left(b):
            Vel = Twist()
            Vel.linear.x = 0
            Vel.angular.z = 1
            pubVel.publish(Vel)
            
        def Right(b):
            Vel = Twist()
            Vel.linear.x = 0
            Vel.angular.z = -1
            pubVel.publish(Vel)
            
        def FrontRight(b):
            Vel = Twist()
            Vel.linear.x = 0.5
            Vel.angular.z = -1
            pubVel.publish(Vel)
        
        def FrontLeft(b):
            Vel = Twist()
            Vel.linear.x = 0.5
            Vel.angular.z = 1
            pubVel.publish(Vel)
        
        def Stop(b):
            Vel = Twist()
            Vel.linear.x = 0
            Vel.angular.z = 0
            pubVel.publish(Vel)
        
        def IncreaseVelocity(b):
            global x_vel, z_vel
            Vel = Twist()
            x_vel += x_vel*0.1
            z_vel += z_vel*0.1
            Vel.linear.x = x_vel
            Vel.angular.z = z_vel
            pubVel.publish(Vel)
            print("Velocity increased:")
            print("Linear velocity = ", x_vel)
            print("Angular velocity = ", z_vel)
            
        def DecreaseVelocity(b):
            global x_vel, z_vel
            Vel = Twist()
            x_vel -= x_vel*0.1
            z_vel -= z_vel*0.1
            Vel.linear.x = x_vel
            Vel.angular.z = z_vel
            pubVel.publish(Vel)
            print("Velocity decreased:")
            print("Linear velocity = ", x_vel)
            print("Angular velocity = ", z_vel)
        
        assistedDrive.observe(setAssisted)
        ButtonFront.on_click(Front)
        ButtonBack.on_click(Back)
        ButtonLeft.on_click(Left)
        ButtonRight.on_click(Right)
        ButtonFrontLeft.on_click(FrontLeft)
        ButtonFrontRight.on_click(FrontRight)
        ButtonStop.on_click(Stop)    
        ButtonIncreaseV.on_click(IncreaseVelocity)
        ButtonDecreaseV.on_click(DecreaseVelocity)

In [12]:
# Definition of the button for the modality to choose
# Autonomous navigation or Manual drive
b1 = widgets.Button(description='AutonomousNavigation')

b2 = widgets.Button(description='RobotNavigation')

output1 = widgets.Output()
output2 = widgets.Output()

b1.on_click(AutonomousNavigation)
b2.on_click(RobotNavigation)

display(output1)
display(output2)

HBox([b1, b2])

Output()

Output()

Robot Navigation!


Checkbox(value=False, description='Assisted drive', indent=False)

Button(description='Front', style=ButtonStyle())

...

Button(description='Stop', style=ButtonStyle())

Button(description='FrontLeft', style=ButtonStyle())

Button(description='FrontRight', style=ButtonStyle())

Button(description='IncreaseV', style=ButtonStyle())

Button(description='DecreaseV', style=ButtonStyle())

In [13]:
## Cell to initialize the node and to define publisher and subscriber
#init node
rospy.init_node("RobSim")
#All the subscriber
pubVel = rospy.Publisher("/cmd_vel", Twist, queue_size = 1)
pubGoal = rospy.Publisher("move_base/goal", MoveBaseActionGoal, queue_size = 1)
pubCancel = rospy.Publisher("move_base/cancel", GoalID, queue_size = 1)
#All the publisher
SubGoal = rospy.Subscriber('/move_base/goal', MoveBaseActionGoal, CurrentGoal)
SubStatus = rospy.Subscriber('/move_base/feedback', MoveBaseActionFeedback, GoalStatus)
SubAvoidColl = rospy.Subscriber('/scan', LaserScan, CollisionAvoidanceCallback)

## Data Visualization ##

In [18]:
%matplotlib widget

class Visualiser:
    
    def __init__(self):
        
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'b-')
        self.x_data, self.y_data = [] , []
        
    def plot_init(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        return self.ln
    
    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)
        
    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln
    
vis = Visualiser()
sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
plt.show(block=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Laser scan
In this cell, the code for the plot of the lasers scan in implemented. What is shown in the plot is what the robot sees with its laser scanner in a frontal view.

In [15]:
import math
class Visualiser:
    def __init__(self):
        
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'r.')
        self.x_plot, self.y_plot = [], []
    
    def plot_init(self):
        self.ax.set_xlim(-15,15)
        self.ax.set_ylim(-15,15)
        return self.ln
        
    def scanner_callback(self, msg):
        self.x_plot = []
        self.y_plot = []
        angle = msg.angle_min
        for i in range(0, len(msg.ranges)):
            
            self.x_plot.append(msg.ranges[i]*math.cos(angle + (90.0*3.1416/180.0)))
            self.y_plot.append(msg.ranges[i]*math.sin(angle + (90.0*3.1416/180.0)))
            angle = angle + msg.angle_increment
            
    def update_plot(self, frame):
        self.ln.set_data(self.x_plot, self.y_plot)
        return self.ln
        

vis = Visualiser()
sub = rospy.Subscriber('/scan', LaserScan, vis.scanner_callback)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func = vis.plot_init)
plt.show(block = True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
fig2, ax2 = plt.subplots()
def ShowPiePlot(b):
    global fig2, ax2    
    ax2.cla()
    labels = 'Reached', 'Not reached'
    sizes = [achieved, notAchieved]    
    ax2.pie(sizes, labels = labels, autopct ='%1.1f%%', shadow = True, startangle = 90)
    ax2.axis('equal') 
    plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
PiePlot = widgets.Button(description='ShowPiePlot')
Histogram = widgets.Button(description='ShowHistogram')

PiePlot.on_click(ShowPiePlot)

display(PiePlot)
 

Button(description='ShowPiePlot', style=ButtonStyle())